#Pre-requisites

In [ ]:
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/terrierteam/pyterrier_t5.git to /tmp/pip-req-build-saugx8go
  Running command git clone -q https://github.com/terrierteam/pyterrier_t5.git /tmp/pip-req-build-saugx8go


In [ ]:
import pickle
from transformers import T5ForConditionalGeneration, T5Tokenizer
import itertools
import torch
torch.cuda.empty_cache()
from google.colab import drive
import os


drive.mount("/content/drive", force_remount=True)

os.environ['IR_DATASETS_HOME'] = "/content/drive/MyDrive/Colab Notebooks/Dissertation/ir_datasets"

print(os.getenv('IR_DATASETS_HOME'))

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Dissertation/ir_datasets


In [ ]:
#Loading eval input and output datasets
with open('/content/drive/MyDrive/Colab Notebooks/Dissertation/inp.pkl', 'rb') as handle:
    inp = pickle.load(handle)

with open('/content/drive/MyDrive/Colab Notebooks/Dissertation/out.pkl', 'rb') as handle:
    out = pickle.load(handle)

In [ ]:
temp_i = inp
temp_o = out

len(inp)

1070091

In [ ]:
inp = inp[:600]
out = out[:600]

len(inp)

600

In [ ]:
#Initializing the tokenizer and creating the input ids
tokenizer = T5Tokenizer.from_pretrained("t5-base")
inp_ids = tokenizer(inp, return_tensors='pt', padding=True).input_ids.cuda()

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


#Defining the model paths

In [ ]:
model_path_baseline=[]
model_path_individual = []
model_path_all_wgt0=[]
model_path_all_wgt1=[]
model_path_rel_wgt=[]

for i in range(10):
  model_path_baseline.append('/content/drive/MyDrive/Colab Notebooks/Dissertation/Models/baseline-'+str(i))
  model_path_individual.append('/content/drive/MyDrive/Colab Notebooks/Dissertation/Models/indi-nrwgt1-'+str(i))
  model_path_rel_wgt.append('/content/drive/MyDrive/Colab Notebooks/Dissertation/Models/relwgt-nrwgt0-'+str(i))
  model_path_all_wgt0.append('/content/drive/MyDrive/Colab Notebooks/Dissertation/Models/batch-allwgt-nrwgt0-'+str(i))
  model_path_all_wgt1.append('/content/drive/MyDrive/Colab Notebooks/Dissertation/Models/batch-allwgt-nrwgt1-'+str(i))

#Evaluation

In [ ]:
result=[]
individual_results = []
for i in range(10):
  #Defining the model
  model = T5ForConditionalGeneration.from_pretrained(model_path_individual[i])
  device = torch.device("cuda")
  model.to(device)

  with torch.no_grad():
    #saving model predictions in result variable
    result = model(input_ids = inp_ids, decoder_input_ids=torch.full((len(inp), 1), model.config.decoder_start_token_id, dtype=torch.long, device=device)).logits
  #Extracting only probabilities of true and false
  result = result[:, 0, (1176, 6136)]
  #COnverting the probabilites into true or false
  individual_results.append(['true' if row[0] > row[1] else 'false' for row in result])
#Saving the predictions to google drive
with open(f'/content/drive/MyDrive/Colab Notebooks/Dissertation/Predictions/indi10.pkl', 'wb') as handle:
    pickle.dump(individual_results, handle)

9
10


In [ ]:
result=[]
individual_results = []
for i in range(10):
  
  model = T5ForConditionalGeneration.from_pretrained(model_path_rel_wgt[i])
  device = torch.device("cuda")
  model.to(device)

    with torch.no_grad():
    #saving model predictions in result variable
    result = model(input_ids = inp_ids, decoder_input_ids=torch.full((len(inp), 1), model.config.decoder_start_token_id, dtype=torch.long, device=device)).logits
  #Extracting only probabilities of true and false
  result = result[:, 0, (1176, 6136)]
  #COnverting the probabilites into true or false
  individual_results.append(['true' if row[0] > row[1] else 'false' for row in result])
#Saving the predictions to google drive
with open(f'/content/drive/MyDrive/Colab Notebooks/Dissertation/Predictions/rel_wgt10.pkl', 'wb') as handle:
  pickle.dump(individual_results, handle)

9
10


In [ ]:
result=[]
individual_results = []
for i in range(10):
  
  model = T5ForConditionalGeneration.from_pretrained(model_path_all_wgt0[i])
  device = torch.device("cuda")
  model.to(device)

    with torch.no_grad():
    #saving model predictions in result variable
    result = model(input_ids = inp_ids, decoder_input_ids=torch.full((len(inp), 1), model.config.decoder_start_token_id, dtype=torch.long, device=device)).logits
  #Extracting only probabilities of true and false
  result = result[:, 0, (1176, 6136)]
  #COnverting the probabilites into true or false
  individual_results.append(['true' if row[0] > row[1] else 'false' for row in result])
#Saving the predictions to google drive
with open(f'/content/drive/MyDrive/Colab Notebooks/Dissertation/Predictions/all_wgt0-10.pkl', 'wb') as handle:
  pickle.dump(individual_results, handle)

9
10


In [ ]:
result=[]
individual_results = []
for i in range(10):
  
  model = T5ForConditionalGeneration.from_pretrained(model_path_all_wgt1[i])
  device = torch.device("cuda")
  model.to(device)

    with torch.no_grad():
    #saving model predictions in result variable
    result = model(input_ids = inp_ids, decoder_input_ids=torch.full((len(inp), 1), model.config.decoder_start_token_id, dtype=torch.long, device=device)).logits
  #Extracting only probabilities of true and false
  result = result[:, 0, (1176, 6136)]
  #COnverting the probabilites into true or false
  individual_results.append(['true' if row[0] > row[1] else 'false' for row in result])
#Saving the predictions to google drive
with open(f'/content/drive/MyDrive/Colab Notebooks/Dissertation/Predictions/all_wgt1-10.pkl', 'wb') as handle:
  pickle.dump(individual_results, handle)

9
10


In [ ]:
result=[]
individual_results = []
for i in range(10):
  
  model = T5ForConditionalGeneration.from_pretrained(model_path_baseine[i])
  device = torch.device("cuda")
  model.to(device)

    with torch.no_grad():
    #saving model predictions in result variable
    result = model(input_ids = inp_ids, decoder_input_ids=torch.full((len(inp), 1), model.config.decoder_start_token_id, dtype=torch.long, device=device)).logits
  #Extracting only probabilities of true and false
  result = result[:, 0, (1176, 6136)]
  #COnverting the probabilites into true or false
  individual_results.append(['true' if row[0] > row[1] else 'false' for row in result])
#Saving the predictions to google drive
with open(f'/content/drive/MyDrive/Colab Notebooks/Dissertation/Predictions/baseline10.pkl', 'wb') as handle:
  pickle.dump(individual_results, handle)

9
10
